## Features Engineering

In [286]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [287]:
# loading data
df = pd.read_csv('../data/train.csv', index_col='id')
print(df.shape)
df.head()

(54273, 12)


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [288]:
df.shape

(54273, 12)

In [289]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
model_year,54273.0,2015.091979,5.588909,1974.0,2012.0,2016.0,2019.0,2024.0
milage,54273.0,72746.175667,50469.490448,100.0,32268.0,66107.0,102000.0,405000.0
price,54273.0,39218.443333,72826.335535,2000.0,15500.0,28000.0,45000.0,2954083.0


In [290]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54273 entries, 0 to 54272
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         54273 non-null  object
 1   model         54273 non-null  object
 2   model_year    54273 non-null  int64 
 3   milage        54273 non-null  int64 
 4   fuel_type     54273 non-null  object
 5   engine        54273 non-null  object
 6   transmission  54273 non-null  object
 7   ext_col       54273 non-null  object
 8   int_col       54273 non-null  object
 9   accident      54273 non-null  object
 10  clean_title   54273 non-null  object
 11  price         54273 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 5.4+ MB


In [291]:
df.nunique()

brand             53
model           1827
model_year        34
milage          3212
fuel_type          7
engine          1061
transmission      46
ext_col          260
int_col          124
accident           2
clean_title        1
price           1481
dtype: int64

In [292]:
# one hot encoding features | unique values count < 10
hot_features = ['fuel_type', 'accident']
drop_features = ['clean_title']             # only have one feature
ordinal_labeling = ['model', 'brand', 'engine', 'transmission', 'ent_col', 'int_col']


In [293]:
patterns = {
    'horsepower': r'(\d+(\.\d+)?(?:HP| Horsepower| HP| Electric Motor))',
    'liters': r'(\d+(\.\d+)?(?:L| Liters| Liter))',
    'cylinders': r'(\d+ (?:Cylinder|Cylinders|Cyl|I\d|V\d|Straight \d))',
    'fuel_type': r'(Gasoline Fuel|Electric Fuel System|Flex Fuel Capability|Gas/Electric Hybrid|Hydrogen Fuel|Plug-In Electric/Gas|Gasoline/Mild Electric Hybrid|Diesel Fuel|Hybrid|Gas)'
}

# Extract features using regex
df['horsepower'] = df['engine'].str.extract(patterns['horsepower']).drop([1], axis=1)
df['liters'] = df['engine'].str.extract(patterns['liters']).drop([1], axis=1)
df['cylinders'] = df['engine'].str.extract(patterns['cylinders'])
df['engine_type'] = df['engine'].str.extract(patterns['fuel_type'])

# Clean up the extracted columns if necessary
df['horsepower'] = df['horsepower'].str.replace('HP', '').str.replace(' Horsepower', '').str.replace(' HP', '').str.replace(' Electric Motor', '').astype('float', errors='ignore')
df['liters'] = df['liters'].str.replace('L', '').str.replace(' Liters', '').str.replace(' Liter', '').str.replace(' iter', '').astype('float', errors='ignore')
df['cylinders'] = df['cylinders'].str.extract(r'(\d+)').astype('int', errors='ignore')

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,liters,cylinders,engine_type
id,,,,,,,,,,,,,,,,
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.0,3.5,6,Gasoline Fuel
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,300.0,3.0,6,Gasoline Fuel
2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.0,4.2,8,Gasoline Fuel
3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.0,3.0,6,Gasoline/Mild Electric Hybrid
4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850,200.0,3.8,6,Gasoline Fuel


In [294]:
# change transmission colums
# Define regex patterns

patterns = {
    'type': r'(A/T|M/T|Automatic|Manual|CVT|DCT|Electronically Controlled Automatic|Variable)',
    'speeds': r'(\d+)-Speed',
    'additional_features': r'(with Auto-Shift|with Overdrive|Dual Shift Mode|Overdrive|Auto-Shift)'
}

# Extract features using regex
df['transmission_type'] = df['transmission'].str.extract(patterns['type'], flags=re.IGNORECASE)
df['speeds'] = df['transmission'].str.extract(patterns['speeds'])
df['additional_features'] = df['transmission'].str.extract(patterns['additional_features'], flags=re.IGNORECASE)

# Fill missing transmission types based on keywords
df['transmission_type'] = df['transmission_type'].fillna(
    df['transmission'].apply(lambda x: 'Automatic' if 'A/T' in x else 'Manual' if 'M/T' in x else 'CVT' if 'CVT' in x else np.NaN)
)

# Convert speeds to integer
# df['speeds'] = df['speeds'].astype('Int64')

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,liters,cylinders,engine_type,transmission_type,speeds,additional_features
id,,,,,,,,,,,,,,,,,,,
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.0,3.5,6,Gasoline Fuel,A/T,10,NaN
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,300.0,3.0,6,Gasoline Fuel,M/T,6,NaN
2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.0,4.2,8,Gasoline Fuel,A/T,6,NaN
3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.0,3.0,6,Gasoline/Mild Electric Hybrid,NaN,NaN,Dual Shift Mode
4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850,200.0,3.8,6,Gasoline Fuel,A/T,NaN,NaN


In [295]:
df.nunique()

brand                    53
model                  1827
model_year               34
milage                 3212
fuel_type                 7
engine                 1061
transmission             46
ext_col                 260
int_col                 124
accident                  2
clean_title               1
price                  1481
horsepower              341
liters                   60
cylinders                 7
engine_type               9
transmission_type         8
speeds                    9
additional_features       4
dtype: int64

In [296]:
# imputing values
from sklearn.impute import SimpleImputer

num_col_impute = ['horsepower', 'liters', 'cylinders']

num_imputer_data = {}
for i in num_col_impute:
    sim = SimpleImputer(strategy='mean')
    df[i] = sim.fit_transform(df[[i]])
    num_imputer_data[i] = sim



In [297]:
# cetegorize features

def ordinal_feature_labeler(df ,features_list, target):
    ''' 
    This function will label categorical features list features to ordinal label.
    '''
    transformed_df = pd.DataFrame({"temp":np.arange(df.shape[0])}, index=df.index)

    transformed_label_data = {}

    def func(x, d):
        try:
            return d[x]
        except:
            return -1

    for feature in features_list:
        ordered_values = df.groupby([feature])[target].median().sort_values().index.to_list()
        n = len(ordered_values)

        value_dict = {i:j for i, j in zip(ordered_values, list(range(n)))}
        # copy the categories values data
        transformed_label_data[feature] = value_dict.copy()

        transformed_df[feature] = df[feature].apply(lambda x: func(x, value_dict))
    
    transformed_df.drop(['temp'], axis=1, inplace=True)

    return transformed_df, transformed_label_data


def label_from_dict(df, label_dict):
    """
    Transforms the given df categorical labels according to given dict labels.
    """
    temp_df = df.copy()

    for feature in label_dict.keys():
        labels = label_dict[feature]
        temp_df[feature] = temp_df[feature].apply(lambda x: labels[x] if x in labels.keys() else len(label_dict) / 2)

    return temp_df





In [298]:
# transforming columns
feature_to_transform = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'engine_type', 'transmission_type']
dont_change = 'model_year milage price'.split()

temp_df, labels = ordinal_feature_labeler(df, feature_to_transform, 'price')

df = label_from_dict(df, labels)


df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,liters,cylinders,engine_type,transmission_type,speeds,additional_features
id,,,,,,,,,,,,,,,,,,,
0,32,1272,2018,74349,3,769,31,60,3,None reported,Yes,11000,375.0,3.5,6.0,3.0,3.0,10,NaN
1,30,384,2007,80000,3,418,9,45,25,None reported,Yes,8250,300.0,3.0,6.0,3.0,0.0,6,NaN
2,27,358,2009,91491,3,265,5,9,6,None reported,Yes,15000,300.0,4.2,8.0,3.0,3.0,6,NaN
3,30,1562,2022,2437,6,860,17,87,43,None reported,Yes,63500,335.0,3.0,6.0,8.0,4.5,NaN,Dual Shift Mode
4,9,223,2001,111000,3,81,4,48,25,None reported,Yes,7850,200.0,3.8,6.0,3.0,3.0,NaN,NaN


In [299]:
# # lebel Encoding 
# from sklearn.preprocessing import OrdinalEncoder

# features_to_encode = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'engine_type', 'transmission_type']



# ordinal_encoding_data = {}
# for feat in features_to_encode:
#     le = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
#     df[feat] = le.fit_transform(df[[feat]])
#     ordinal_encoding_data[feat] = le
#     del(le)

# df.head()

In [300]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(handle_unknown='ignore')
temp = onehot.fit_transform(df[['additional_features']])
onehot.categories_

df[onehot.get_feature_names_out()] = temp.toarray()

onehot2 = OneHotEncoder(handle_unknown='ignore')
temp2 = onehot2.fit_transform(df[['accident']])
df[onehot2.get_feature_names_out()] = temp2.toarray()

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,transmission_type,speeds,additional_features,additional_features_Dual Shift Mode,additional_features_Overdrive,additional_features_with Auto-Shift,additional_features_with Overdrive,additional_features_nan,accident_At least 1 accident or damage reported,accident_None reported
id,,,,,,,,,,,,,,,,,,,,,
0,32,1272,2018,74349,3,769,31,60,3,None reported,...,3.0,10,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,30,384,2007,80000,3,418,9,45,25,None reported,...,0.0,6,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,27,358,2009,91491,3,265,5,9,6,None reported,...,3.0,6,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,30,1562,2022,2437,6,860,17,87,43,None reported,...,4.5,NaN,Dual Shift Mode,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,9,223,2001,111000,3,81,4,48,25,None reported,...,3.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [301]:
df.drop(['speeds', 'accident', 'clean_title', 'additional_features', 'additional_features_nan'], axis=1, inplace=True)

df['transmission_type'] = df.transmission_type.replace(np.nan, -1)
df['engine_type'] = df.engine_type.replace(np.nan, -1)

In [302]:
df.isnull().sum()

brand                                              0
model                                              0
model_year                                         0
milage                                             0
fuel_type                                          0
engine                                             0
transmission                                       0
ext_col                                            0
int_col                                            0
price                                              0
horsepower                                         0
liters                                             0
cylinders                                          0
engine_type                                        0
transmission_type                                  0
additional_features_Dual Shift Mode                0
additional_features_Overdrive                      0
additional_features_with Auto-Shift                0
additional_features_with Overdrive            

## transforming test data

In [303]:
test_df = pd.read_csv("../data/test.csv", index_col='id')
print(test_df.shape)
test_df.head()

(36183, 11)


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
id,,,,,,,,,,,
54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [304]:
# making patterns
patterns = {
    'horsepower': r'(\d+(\.\d+)?(?:HP| Horsepower| HP| Electric Motor))',
    'liters': r'(\d+(\.\d+)?(?:L| Liters| Liter))',
    'cylinders': r'(\d+ (?:Cylinder|Cylinders|Cyl|I\d|V\d|Straight \d))',
    'fuel_type': r'(Gasoline Fuel|Electric Fuel System|Flex Fuel Capability|Gas/Electric Hybrid|Hydrogen Fuel|Plug-In Electric/Gas|Gasoline/Mild Electric Hybrid|Diesel Fuel|Hybrid|Gas)'
}

# Extract features using regex
test_df['horsepower'] = test_df['engine'].str.extract(patterns['horsepower']).drop([1], axis=1)
test_df['liters'] = test_df['engine'].str.extract(patterns['liters']).drop([1], axis=1)
test_df['cylinders'] = test_df['engine'].str.extract(patterns['cylinders'])
test_df['engine_type'] = test_df['engine'].str.extract(patterns['fuel_type'])

# Clean up the extracted columns if necessary
test_df['horsepower'] = test_df['horsepower'].str.replace('HP', '').str.replace(' Horsepower', '').str.replace(' HP', '').str.replace(' Electric Motor', '').astype('float', errors='ignore')
test_df['liters'] = test_df['liters'].str.replace('L', '').str.replace(' Liters', '').str.replace(' Liter', '').str.replace(' iter', '').astype('float', errors='ignore')
test_df['cylinders'] = test_df['cylinders'].str.extract(r'(\d+)').astype('int', errors='ignore')

test_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,horsepower,liters,cylinders,engine_type
id,,,,,,,,,,,,,,,
54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes,302.0,3.5,6,Gasoline Fuel
54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes,275.0,3.5,6,Gasoline Fuel
54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes,241.0,2.0,4,Gasoline Fuel
54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes,518.0,5.0,8,Gasoline Fuel
54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,335.0,3.0,6,Gasoline Fuel


In [305]:
test_df.isnull().sum()


brand              0
model              0
model_year         0
milage             0
fuel_type          0
engine             0
transmission       0
ext_col            0
int_col            0
accident           0
clean_title        0
horsepower      2606
liters           364
cylinders       2712
engine_type     2535
dtype: int64

In [306]:
# imputing values in test_df
num_col_impute = ['horsepower', 'liters', 'cylinders']

# num_imputer_data = {}
for i in num_col_impute:
    sim = num_imputer_data[i]
    test_df[i] = sim.transform(test_df[[i]])
    
test_df.isnull().sum()


brand              0
model              0
model_year         0
milage             0
fuel_type          0
engine             0
transmission       0
ext_col            0
int_col            0
accident           0
clean_title        0
horsepower         0
liters             0
cylinders          0
engine_type     2535
dtype: int64

In [307]:
patterns = {
    'type': r'(A/T|M/T|Automatic|Manual|CVT|DCT|Electronically Controlled Automatic|Variable)',
    'speeds': r'(\d+)-Speed',
    'additional_features': r'(with Auto-Shift|with Overdrive|Dual Shift Mode|Overdrive|Auto-Shift)'
}

# Extract features using regex
test_df['transmission_type'] = test_df['transmission'].str.extract(patterns['type'], flags=re.IGNORECASE)
test_df['speeds'] = test_df['transmission'].str.extract(patterns['speeds'])
test_df['additional_features'] = test_df['transmission'].str.extract(patterns['additional_features'], flags=re.IGNORECASE)

# Fill missing transmission types based on keywords
test_df['transmission_type'] = test_df['transmission_type'].fillna(
    test_df['transmission'].apply(lambda x: 'Automatic' if 'A/T' in x else 'Manual' if 'M/T' in x else 'CVT' if 'CVT' in x else np.NaN)
)

# Convert speeds to integer
# test_df['speeds'] = test_df['speeds'].astype('Int64')

test_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,horsepower,liters,cylinders,engine_type,transmission_type,speeds,additional_features
id,,,,,,,,,,,,,,,,,,
54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes,302.0,3.5,6.0,Gasoline Fuel,A/T,NaN,NaN
54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes,275.0,3.5,6.0,Gasoline Fuel,A/T,8,NaN
54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes,241.0,2.0,4.0,Gasoline Fuel,A/T,7,NaN
54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes,518.0,5.0,8.0,Gasoline Fuel,NaN,NaN,Dual Shift Mode
54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,335.0,3.0,6.0,Gasoline Fuel,A/T,8,NaN


In [308]:
# features_to_encode = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'engine_type', 'transmission_type']

# # ordinal_encoding_data = {}
# for feat in features_to_encode:
#     le = ordinal_encoding_data[feat]
#     test_df[feat] = le.transform(test_df[[feat]])
#     del(le)

# test_df.head()

test_df = label_from_dict(test_df, labels)


In [309]:
# onehot = OneHotEncoder(handle_unknown='ignore')
test_temp = onehot.fit_transform(test_df[['additional_features']])
onehot.categories_

test_df[onehot.get_feature_names_out()] = test_temp.toarray()

# onehot2 = OneHotEncoder(handle_unknown='ignore')
test_temp2 = onehot2.fit_transform(test_df[['accident']])
test_df[onehot2.get_feature_names_out()] = test_temp2.toarray()

test_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,transmission_type,speeds,additional_features,additional_features_Dual Shift Mode,additional_features_Overdrive,additional_features_with Auto-Shift,additional_features_with Overdrive,additional_features_nan,accident_At least 1 accident or damage reported,accident_None reported
id,,,,,,,,,,,,,,,,,,,,,
54273,34.0,694.0,2014,73000,3,437.0,4.0,48.0,6.0,None reported,...,3.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54274,25.0,613.0,2015,128032,3,143.0,19.0,16.0,25.0,None reported,...,3.0,8,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54275,34.0,989.0,2015,51983,3,534.0,15.0,60.0,54.0,None reported,...,3.0,7,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54276,36.0,1506.0,2018,29500,3,894.0,17.0,48.0,54.0,At least 1 accident or damage reported,...,4.5,NaN,Dual Shift Mode,1.0,0.0,0.0,0.0,0.0,1.0,0.0
54277,30.0,1479.0,2020,90000,3,736.0,19.0,48.0,25.0,At least 1 accident or damage reported,...,3.0,8,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [310]:
test_df.isnull().sum()

brand                                                  0
model                                                  0
model_year                                             0
milage                                                 0
fuel_type                                              0
engine                                                 0
transmission                                           0
ext_col                                                0
int_col                                                0
accident                                               0
clean_title                                            0
horsepower                                             0
liters                                                 0
cylinders                                              0
engine_type                                            0
transmission_type                                      0
speeds                                             16706
additional_features            

In [311]:
test_df.drop(['speeds', 'accident', 'clean_title', 'additional_features', 'additional_features_nan'], axis=1, inplace=True)

test_df['transmission_type'] = test_df.transmission_type.replace(np.nan, -1)
test_df['engine_type'] = test_df.engine_type.replace(np.nan, -1)

In [312]:
test_df.isnull().sum()

brand                                              0
model                                              0
model_year                                         0
milage                                             0
fuel_type                                          0
engine                                             0
transmission                                       0
ext_col                                            0
int_col                                            0
horsepower                                         0
liters                                             0
cylinders                                          0
engine_type                                        0
transmission_type                                  0
additional_features_Dual Shift Mode                0
additional_features_Overdrive                      0
additional_features_with Auto-Shift                0
additional_features_with Overdrive                 0
accident_At least 1 accident or damage reporte

In [313]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,price,...,liters,cylinders,engine_type,transmission_type,additional_features_Dual Shift Mode,additional_features_Overdrive,additional_features_with Auto-Shift,additional_features_with Overdrive,accident_At least 1 accident or damage reported,accident_None reported
id,,,,,,,,,,,,,,,,,,,,,
0,32,1272,2018,74349,3,769,31,60,3,11000,...,3.5,6.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0
1,30,384,2007,80000,3,418,9,45,25,8250,...,3.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,27,358,2009,91491,3,265,5,9,6,15000,...,4.2,8.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0
3,30,1562,2022,2437,6,860,17,87,43,63500,...,3.0,6.0,8.0,4.5,1.0,0.0,0.0,0.0,0.0,1.0
4,9,223,2001,111000,3,81,4,48,25,7850,...,3.8,6.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0


In [314]:
df.isnull().sum()


brand                                              0
model                                              0
model_year                                         0
milage                                             0
fuel_type                                          0
engine                                             0
transmission                                       0
ext_col                                            0
int_col                                            0
price                                              0
horsepower                                         0
liters                                             0
cylinders                                          0
engine_type                                        0
transmission_type                                  0
additional_features_Dual Shift Mode                0
additional_features_Overdrive                      0
additional_features_with Auto-Shift                0
additional_features_with Overdrive            

In [315]:
test_df.isnull().sum()


brand                                              0
model                                              0
model_year                                         0
milage                                             0
fuel_type                                          0
engine                                             0
transmission                                       0
ext_col                                            0
int_col                                            0
horsepower                                         0
liters                                             0
cylinders                                          0
engine_type                                        0
transmission_type                                  0
additional_features_Dual Shift Mode                0
additional_features_Overdrive                      0
additional_features_with Auto-Shift                0
additional_features_with Overdrive                 0
accident_At least 1 accident or damage reporte

In [316]:
print(df.shape)
print(test_df.shape)


(54273, 21)
(36183, 20)


In [317]:
df['model_year'] = 2024 - df['model_year']
test_df['model_year'] = 2024 - test_df['model_year']
df['milage_per_year'] = (df['milage'] / df['model_year']).round(2)
test_df['milage_per_year'] = (test_df['milage'] / test_df['model_year']).round(2)

col_to_sum = [
    'brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col',
    'engine_type', 'transmission_type', 'liters',
    'additional_features_Dual Shift Mode', 'additional_features_Overdrive',
    'additional_features_with Auto-Shift',
    'additional_features_with Overdrive',
    'accident_At least 1 accident or damage reported',
    'accident_None reported'#, 'milage_per_year'
    ]

df['score'] = df[col_to_sum].sum(axis=1) # - df['model_year']

In [318]:
df.to_csv("../data/transformed_train.csv", index=True)
test_df.to_csv("../data/transformed_test.csv", index=True)


In [319]:
df.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'price', 'horsepower', 'liters',
       'cylinders', 'engine_type', 'transmission_type',
       'additional_features_Dual Shift Mode', 'additional_features_Overdrive',
       'additional_features_with Auto-Shift',
       'additional_features_with Overdrive',
       'accident_At least 1 accident or damage reported',
       'accident_None reported', 'milage_per_year', 'score'],
      dtype='object')

In [320]:
df.sample(5).T
# col_to_sum = [
#     'brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col int_col',
#     'engine_type', 'transmission_type', 'liters',
#     'additional_features_Dual Shift Mode', 'additional_features_Overdrive',
#     'additional_features_with Auto-Shift',
#     'additional_features_with Overdrive',
#     'accident_At least 1 accident or damage reported',
#     'accident_None reported', 'milage_per_year'
#     ]

id,11958,7466,25555,51775,39018
brand,34.000000,14.00,33.00,19.00,36.0
model,1705.000000,186.00,665.00,896.00,1624.0
model_year,2.000000,17.00,19.00,3.00,4.0
milage,8940.000000,100600.00,97500.00,224000.00,110000.0
fuel_type,3.000000,3.00,3.00,3.00,3.0
engine,976.000000,51.00,455.00,538.00,894.0
transmission,21.000000,4.00,0.00,4.00,4.0
ext_col,209.000000,60.00,24.00,48.00,48.0
int_col,25.000000,3.00,6.00,25.00,54.0
price,84620.000000,10000.00,7200.00,38500.00,17950.0
